<a href="https://colab.research.google.com/github/ammarisme/export-forecast-NR/blob/master/Attention_NRNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.5.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Using cached https://pytorch-geometric.com/whl/torch-1.5.0/torch_scatter-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Using cached https://pytorch-geometric.com/whl/torch-1.5.0/torch_cluster-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-sparse 0.6.4
    Uninstalling torch-sparse-0.6.4:
      Successfully uninstalled torch-sparse-0.6.4
  Found existing installation: torch-scatter 2.0.4
    Uninstalling torch-scatter-2.0.4:
      Successfully uninstalled torch-scatter-2.0.4
  Found existing installation: torch-cluster 1.5.4
    Uninstalling torch-cluster-1.5.4:
      Successfully uninstalled torch-cluster-1.5.4


In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os

import time
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader, InMemoryDataset

import numpy as np
from numpy import isnan
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from datetime import datetime


device = torch.device("cuda:0")           
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device, torch.cuda.is_available())
print('number of cuda devices : ', torch.cuda.device_count())
print('memory allocated : ', torch.cuda.memory_allocated(device))

cuda:0 True
number of cuda devices :  1
memory allocated :  110097920


In [0]:
from google.colab import drive
import os.path
from os import path
import sys
from os import listdir
from os.path import isfile, join

drive.flush_and_unmount()
drive.mount('/content/drive')
PATH = '/content/drive/My Drive/rubber'
INPUT_ROOT = PATH+'/input_mix/test'
sys.path.append(PATH)

Mounted at /content/drive


In [0]:
"""
Common function
"""
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp
  

def save_model_parameters(model_parameters_dict, models = None, fileName=None):
  if fileName == None:
    fileName = 'model_parameters_'+str(np.round(np.random.randn(1000,200)[0][0]*1000))

  model_summary = pd.DataFrame(columns =[])
  for i in sorted(model_parameters_dict.keys()):
    param_size = 0
    if models != None:
      param_size = get_n_params(models[i][0])
      param_size += get_n_params(models[i][1])
    row = pd.Series({
                        'model_id':i,
                        'encoder repeats' : model_parameters_dict[i][0]['repeats'],
                        'decoder repeats' : model_parameters_dict[i][1]['repeats'],
                        'parameters' :param_size
    })
    row_df = pd.DataFrame([row], index = [i])
    model_summary = pd.concat([model_summary, row_df])
  model_summary.to_csv(fileName)

#Generate the list of repeat count parameters list

#generate_model_parameters(generate_repeat_counts())

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

def save_dict(dictionary, filename):
  np.save(filename, dictionary)

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points, points2):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.plot(points)
    plt.plot(points2)

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def latest_model(mypath):
  model = Encoder(**model_configurations[0][0])
  fnames= []
  for fname in listdir(mypath):
    if "cleaned_non_window" in fname:
      fnames.append(int(fname.split('_')[0]))
  print('Model path: ', mypath+'/'+str(min(fnames))+'_cleaned_non_window.model')
  model = torch.load(mypath+'/'+str(min(fnames))+'_cleaned_non_window.model')
  return model

def get_latest_model(mypath):
  model = Encoder(**model_configurations[0][0])
  model = torch.load(mypath+'/latest.model')
  optimizer = torch.load(mypath+'/latest.optim')
  return model, optimizer

  

In [0]:
class Seq2SeqDataSet(InMemoryDataset):
    def __init__(self, root, input_sequence, output_sequence, transform=None, pre_transform=None):
        super(Seq2SeqDataSet, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_dir(self):
      if os.path.exists(self.root+PROCESSED_DIR):
        return self.root+CLEANED_DIR #'/cleaned'
      else:
        os.mkdir(self.root+PROCESSED_DIR)
        return self.root+CLEANED_DIR#'/cleaned'
        
    @property
    def processed_dir(self):
      if os.path.exists(self.root+PROCESSED_DIR):
        return self.root+PROCESSED_DIR
      else:
        os.mkdir(self.root+PROCESSED_DIR)
        return self.root+PROCESSED_DIR

    @property
    def raw_file_names(self):
      mypath = self.raw_dir
      filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]
      return filenames

    @property
    def processed_file_names(self):
        return ['processed.dt']

    def download(self):
        pass
    
    def process(self):
      output_size , output_sequence_len, input_sequence_len= configuration['output_size'],\
      configuration['output_sequence_len'],\
      configuration['input_sequence_len'],
        
      data_list = []

      for raw_path in self.raw_paths:
        print(raw_path)
        df = pd.read_csv(raw_path)
        for synthetic_seq in df['synthesis_seq'].unique():
          synthetic_data = df[df['synthesis_seq']==synthetic_seq]

          country_data = synthetic_data[features]
                      
          #del country_data['synthesis_seq']
          x_columns = [feature for feature in features if feature != 'target']
          country_data_i = country_data[x_columns][:-output_sequence_len]
          country_data_array = country_data_i.to_numpy()
          feature_length = len(x_columns)
          x = country_data_array#[:feature_length].T

          country_data_o = country_data[input_sequence_len:]
          #country_data_array = country_data_array.reshape(feature_length,len(country_data_i))
          country_data_array_y = np.array([country_data_o['target'].to_numpy()])
          country_data_array_y = country_data_array_y.reshape(output_size,len(country_data_o))
          y = country_data_array_y[:output_size].T

          sets =0
          x_list = []
          dict_x = dict()
          for i in range(input_sequence_len):
            array_len = (len(x)-i) - ((len(x)  - i)%input_sequence_len)+i
            if array_len <= 0:
              print('skipping')
              continue
            sets = int( array_len/ input_sequence_len)
            if sets <= 0:
              print('skipping')
              continue
            #print(len(x))
            #print('input seq : ', i , ' ', array_len , ' ',array_len-i , ' number of sets : ', sets)
            x_temp = x[i:array_len].T.reshape(sets, feature_length, input_sequence_len)
            uniq_keys = np.array([i+k*input_sequence_len for k in range(sets)])
            x_temp = x_temp.reshape(feature_length,sets,input_sequence_len)
            arrays_split = np.hsplit(x_temp,sets)
            dict_x.update(dict(zip(uniq_keys, arrays_split)))

          dict_y = dict()
          y_list = []
          for i in range(output_sequence_len):
            array_len_y = (len(y)-i) - ((len(y)  - i)%output_sequence_len)+i
            if array_len_y <= 0:
              continue
            sets = int(array_len_y / output_sequence_len)
            if sets <= 0:
              continue
            y_temp = y[i:array_len_y].T.reshape(sets, output_size, output_sequence_len)
            #uniq_keys = np.array([i+(output_sequence_len*k) for k in range(output_sequence_len)])
            uniq_keys = np.array([i+k*output_sequence_len for k in range(sets)])
            y_temp = y_temp.reshape(output_size,sets,output_sequence_len)
            arrays_split = np.hsplit(y_temp,sets)
            dict_y.update(dict(zip(uniq_keys, arrays_split)))

          temp_x_list = []
          mean = np.mean(country_data[x_columns].to_numpy(), axis=0).T
          std = np.std(country_data[x_columns].to_numpy(), axis=0).T
          #print(sorted(dict_x.keys()))
          for i in sorted(dict_x.keys()):
            x = dict_x[i].squeeze()
            #print(x.T[0][12])
            x = (x.T - mean) / std
            where_are_NaNs = isnan(x)
            x[where_are_NaNs] = 0
            temp_x_list.append(x)

          temp_y_list  = [dict_y[i].T for i in sorted(dict_y.keys())]

          #_country_code,popData2018
          xy_list = [Data(x = torch.from_numpy(features).type(torch.FloatTensor).squeeze()) for features in temp_x_list]
          
          mean_y = np.mean(y)
          std_y = np.std(y)
          for i in sorted(dict_y.keys()):
            temp_y_list[i] = temp_y_list[i].squeeze()
            norm_y = (temp_y_list[i] -mean_y)/std_y
            xy_list[i].y = torch.from_numpy(norm_y).squeeze()

          data_list += xy_list
        print('processed : '+ raw_path)
      data, slices = self.collate(data_list)
      torch.save((data, slices), self.processed_paths[0])


In [0]:
def get_selected_features():
  original_dataset = pd.read_csv(INPUT_ROOT+'/validation/cleaned/all_hs_codes_combined.csv')
  features = ['target']
  #features.extend([column for column in original_dataset.columns if (("4001" in column) or ("4003" in column) or ("4002" in column)) and (("China" in column) or ("Japan" in column))])
  features.extend([column for column in original_dataset.columns if ("target" not in column)])
  return features

drive.flush_and_unmount()
drive.mount('/content/drive')

INPUT_ROOT = PATH+'/input_mix/test'

features = get_selected_features()
print(features)
configuration = {
    'input_sequence_len' : 12,
    'output_sequence_len' : 12,
    'training_batch_size' : 2048,
    'training_dataset_length' :32768,
    'validation_batch_size' : 2,
    'yhat_size' : 1,
    'feature_len' : len(features)-1,
    'output_size' : 1,
}

DATA_TAG = "window_hs_seq2seq_"+str(configuration['input_sequence_len'])+'_'+str(configuration['output_sequence_len'])
PROCESSED_DIR = '/processed_'+DATA_TAG
CLEANED_DIR =  '/cleaned'


validation_dataset = Seq2SeqDataSet(INPUT_ROOT+'/validation', configuration['input_sequence_len'], configuration['output_sequence_len'])
validation_dataset = validation_dataset.shuffle()
validation_dataloader = DataLoader(validation_dataset,batch_size=configuration['validation_batch_size'])

print(validation_dataset[0])

Mounted at /content/drive
['target', 'Albania_4001', 'Albania_4002', 'Albania_4003', 'Albania_4004', 'Albania_4005', 'Albania_4006', 'Albania_4007', 'Albania_4008', 'Albania_4009', 'Albania_4010', 'Albania_4011', 'Albania_4012', 'Albania_4013', 'Albania_4014', 'Albania_4015', 'Albania_4016', 'Albania_4017', 'Algeria_4001', 'Algeria_4002', 'Algeria_4003', 'Algeria_4004', 'Algeria_4005', 'Algeria_4006', 'Algeria_4007', 'Algeria_4008', 'Algeria_4009', 'Algeria_4010', 'Algeria_4011', 'Algeria_4012', 'Algeria_4013', 'Algeria_4014', 'Algeria_4015', 'Algeria_4016', 'Algeria_4017', 'Angola_4001', 'Angola_4002', 'Angola_4003', 'Angola_4004', 'Angola_4005', 'Angola_4006', 'Angola_4007', 'Angola_4008', 'Angola_4009', 'Angola_4010', 'Angola_4011', 'Angola_4012', 'Angola_4013', 'Angola_4014', 'Angola_4015', 'Angola_4016', 'Angola_4017', 'Antigua and Barbuda_4001', 'Antigua and Barbuda_4002', 'Antigua and Barbuda_4003', 'Antigua and Barbuda_4004', 'Antigua and Barbuda_4005', 'Antigua and Barbuda_400

In [0]:
class RNNEncoder(nn.Module):
    def __init__(self, rnn_num_layers=1, input_feature_len=1, sequence_len=168, hidden_size=100, bidirectional=False):
        super(RNNEncoder, self).__init__()
        self.sequence_len = sequence_len
        self.hidden_size = hidden_size
        self.input_feature_len = input_feature_len
        self.num_layers = rnn_num_layers
        self.rnn_directions = 2 if bidirectional else 1
        self.lstm = nn.LSTM(
            num_layers = rnn_num_layers,
            input_size=input_feature_len,
            hidden_size=hidden_size,
            batch_first=True,
            bidirectional=bidirectional
        )
        
    def forward(self, input_seq):
      """
      if torch.cuda.is_available():
        print('cuda available')
        ht = torch.zeros(self.num_layers * self.rnn_directions, input_seq.size(0) , self.hidden_size).cuda()
        ct = torch.zeros(self.num_layers * self.rnn_directions, input_seq.size(0) , self.hidden_size).cuda()
      else:
        ht = torch.zeros(self.num_layers * self.rnn_directions, input_seq.size(0) , self.hidden_size)
        ct = torch.zeros(self.num_layers * self.rnn_directions, input_seq.size(0) , self.hidden_size)
        """
      ht = torch.zeros(self.num_layers * self.rnn_directions, input_seq.size(0) , self.hidden_size).to(device)
      ct = torch.zeros(self.num_layers * self.rnn_directions, input_seq.size(0) , self.hidden_size).to(device)

      if input_seq.ndim < 3:
          input_seq.unsqueeze_(2)
      lstm_out, ht_ct= self.lstm(input_seq, (ht, ct))
      if self.rnn_directions > 1:
          lstm_out = lstm_out.view(input_seq.size(0), self.sequence_len, self.rnn_directions, self.hidden_size)
          lstm_out = torch.sum(lstm_out, axis=2)
      return lstm_out, ht_ct[0].squeeze(0), ht_ct[1].squeeze(0)
   
class AttentionDecoderCell(nn.Module):
    def __init__(self, input_feature_len, hidden_size, sequence_len, decoder_output_size):
        super(AttentionDecoderCell, self).__init__()
        # attention - inputs - (decoder_inputs, prev_hidden)
        self.attention_linear = nn.Linear(hidden_size + input_feature_len, sequence_len)
        # attention_combine - inputs - (decoder_inputs, attention * encoder_outputs)
        self.decoder_rnn_cell = nn.LSTMCell(
            input_size=hidden_size,
            hidden_size=hidden_size,
        )
        self.out = nn.Linear(hidden_size, decoder_output_size)
        
    def forward(self, encoder_output, prev_hidden, prev_ct, y):
      attention_input = torch.cat((prev_hidden, y), axis=1)
      attention_weights = F.softmax(self.attention_linear(attention_input)).unsqueeze(1)
      attention_combine = torch.bmm(attention_weights, encoder_output).squeeze(1)
      rnn_hidden, prev_ct = self.decoder_rnn_cell(attention_combine, (prev_hidden, prev_ct))
      output = self.out(rnn_hidden)
      return output, rnn_hidden, prev_ct
    

class EncoderDecoderWrapper():
    def __init__(self, encoder, decoder_cell, optimizers = None,  output_size=3, teacher_forcing=0.3, learning_rate = 0.01):
        super().__init__()
        self.encoder = encoder
        self.decoder_cell = decoder_cell
        self.output_size = output_size
        self.teacher_forcing = teacher_forcing
        
        if optimizers == None:
          self.optimizers_init(learning_rate)
        
    def train(self):
        self.encoder.train()
        self.decoder_cell.train()
        
    def eval(self):
        self.encoder.eval()
        self.decoder_cell.eval()
        
    def state_dict(self):
        return {
            'encoder': self.encoder.state_dict(),
            'decoder_cell': self.decoder_cell.state_dict()
        }
    
    def load_state_dict(self, state_dict):
        self.encoder.load_state_dict(state_dict['encoder'])
        self.decoder_cell.load_state_dict(state_dict['decoder_cell'])

    def __call__(self, input_seq, batch_size, yb=None):
      input_seq = input_seq.to(device)
      encoder_output, encoder_hidden, encoder_ct = self.encoder(input_seq)
      prev_hidden = encoder_hidden
      prev_ct = encoder_ct
      """
      if torch.cuda.is_available():
          outputs = torch.zeros(1, self.output_size, device='cuda')
      else:
          outputs = torch.zeros(1, self.output_size)
          """
      outputs = torch.zeros(batch_size, self.output_size).to(device)
      y_prev = torch.zeros(batch_size,1).to(device)
      for i in range(self.output_size):
          if (yb is not None) and (i > 0) and (torch.rand(1) < self.teacher_forcing):
              y_prev = yb[:, i].unsqueeze(1)
          rnn_output, prev_hidden, prev_ct = self.decoder_cell(encoder_output, prev_hidden, prev_ct, y_prev)
          y_prev = rnn_output
          outputs[:, i] = rnn_output.squeeze(1)
      return outputs
    
    def step(self):
      self.optimizers[0].step()
      self.optimizers[1].step()

    def optimizers_init(self, learning_rate):
      enc_optim = torch.optim.Adam(self.encoder.parameters(), lr = learning_rate)
      dec_optim = torch.optim.Adam(self.decoder_cell.parameters(), lr = learning_rate)
      self.optimizers = (
          enc_optim, dec_optim
      )

    def optimizers_zero_grad(self):
      self.optimizers[0].zero_grad()
      self.optimizers[1].zero_grad()

In [0]:
configuration = {
    'input_sequence_len' : 12,
    'output_sequence_len' : 12,
    'training_batch_size' : 1,
    'training_dataset_length' :32768,
    'validation_batch_size' : 1,
    'yhat_size' : 1,
    'feature_len' :len(features)-1 ,
    'output_size' : 1,
}

In [0]:
def train_rnn(input_tensor, target_tensor, model, criterion,batch_size):
  input_tensor = input_tensor.type(torch.FloatTensor)
  target_tensor= target_tensor.type(torch.FloatTensor)
  target_tensor = target_tensor.to(device)
  model.train()
  model.optimizers_zero_grad()
  loss = 0
  yhat = model(input_tensor, batch_size) #remove in batch training mode
  loss += criterion(yhat.squeeze(),target_tensor.squeeze() )
  loss.backward()
  model.step()
  return loss.item() / configuration['input_sequence_len']

In [0]:
def evaluate_rnn(input_tensor, target_tensor, model, criterion,batch_size):
  with torch.no_grad():
    input_tensor = input_tensor.type(torch.FloatTensor)
    target_tensor= target_tensor.type(torch.FloatTensor)
    target_tensor = target_tensor.to(device)
    model.eval()
    val_loss = 0
    yhat = model(input_tensor, batch_size) #remove in batch training mode
    val_loss += criterion(yhat.squeeze(),target_tensor.squeeze() )
  return val_loss.item() / configuration['input_sequence_len']

In [0]:
import copy

def train_batch(dataloader, model, criterion):
  loss = 0
  for batch in dataloader:
    batch.x = batch.x.cuda(device=0)
    batch.y = batch.y.cuda(device=0)
    batch_size = int(len(batch.x)/configuration['input_sequence_len'])
    loss += (train_rnn(batch.x.view(batch_size, configuration['input_sequence_len'] , configuration['feature_len']),
                       batch.y.view(batch_size, configuration['output_sequence_len']), model,
                       criterion, batch_size))/ batch_size
  return loss / len(dataloader)

def batch_evaluate(dataloader, model, criterion, output_sequence_len):
  loss = 0
  for batch in dataloader:
    batch.x = batch.x.to(device)
    batch.y = batch.y.to(device)
    batch_size = int(len(batch.x)/configuration['input_sequence_len'])
    loss += (evaluate_rnn(batch.x.view(batch_size, configuration['input_sequence_len'] , configuration['feature_len']),
                          batch.y.view(batch_size, configuration['output_sequence_len']), model,
                     criterion, batch_size))/ batch_size
  return loss / len(dataloader)

def trainValidationIters(train_dataloader,validation_dataloader, model, 
                        criterion ,n_iters, output_sequence_len,input_sequence_len,
                        current_model_dir, model_id,session_id,print_every=10, plot_every=100,save_every = 100,check_validation_every=1000,id="train_id_"):

  start = time.time()
  plot_losses = []
  
  # Reset every print_every # Reset every plot_every
  print_training_loss_total = 0
  plot_training_loss_total = 0
  print_validation_loss_total = 0
  plot_validation_loss_total = 0
  save_validation_loss_total = 0
  save_training_loss_total = 0
      
  losses = []
  models_temp = dict()
  optim_temp = dict()
  for iter in range(1, n_iters + 1):
    training_loss = train_batch(train_dataloader,model, criterion )
    validation_loss = batch_evaluate(validation_dataloader, model,
                                      criterion,configuration['output_sequence_len'] )

    models_temp[validation_loss] = copy.deepcopy(model) #Save the model

    print_training_loss_total += training_loss
    plot_training_loss_total += training_loss
    print_validation_loss_total += validation_loss
    plot_validation_loss_total += validation_loss
    
    save_training_loss_total += training_loss
    save_validation_loss_total += validation_loss
    
    if iter % print_every == 0:
        print_training_loss_avg = print_training_loss_total / print_every
        print_training_loss_total = 0
        print_validation_loss_avg = print_validation_loss_total / print_every
        print_validation_loss_total = 0
        print('%s (%d %d%%) training:%.15f validation:%.15f' % (timeSince(start, iter / n_iters),
                                      iter, iter / n_iters * 100, print_training_loss_avg,print_validation_loss_avg, ))
        print('train/validation difference : ',print_validation_loss_avg-print_training_loss_avg)
      
    if iter % save_every == 0:
      save_training_loss_avg = save_training_loss_total / save_every
      save_validation_loss_avg = plot_validation_loss_total / save_every
      #'arch': args.arch,
      
    losses.append([training_loss, validation_loss])

    if iter % check_validation_every == 0 and iter >= check_validation_every:
      x = [0 , 1]
      y = np.array(losses).T[1][-1*check_validation_every:].tolist()
      y = [y[-1*check_validation_every:][0], y[-1*check_validation_every:][-1]]
      m,b = np.polyfit(x, y, 1)
      if m > 0 :
        print(y)
        print('Terminating the training at :',iter , )
        break
    torch.cuda.empty_cache()
  return losses

In [0]:
#Run all the candidate models
def train_models( model,training_dataloader,learning_rate = 0.001,epochs = 2500, id= "0"):
  #for i in model_configurations:
  #  #instantiate the models
  i = 1
  if i != 0 :
    print('skipping', i)
    #continue
  current_model_dir =  MODEL_DIR +'/'+str(i)
  if os.path.exists(current_model_dir) == False:
    os.mkdir(current_model_dir)
  losses = trainValidationIters(training_dataloader,validation_dataloader,
                    model,
                    criterion, n_iters = epochs,output_sequence_len=configuration['output_sequence_len'],
                    input_sequence_len=configuration['input_sequence_len'],
                    current_model_dir=current_model_dir,  model_id = i,session_id = session_id,
                    print_every=10, save_every = 100,check_validation_every=3000, id=id)
  
  print('Model ',str(i), ': trained and evaluated')
  return model

In [0]:
INPUT_ROOT = PATH+'/input_mix/test'

In [0]:
#Create a new directory to store model results
MODEL_PREFIX = '_hs'
MODEL_DIR =  PATH + '/models'+MODEL_PREFIX
dirs = []
for f in listdir(MODEL_DIR):
  if "run" in f:
    dirs.append(int(f[3]))
if len(dirs) > 0:
  if os.path.exists(MODEL_DIR+"/run"+str(max(dirs)+1)) == False:
    os.mkdir(MODEL_DIR+"/run"+str(max(dirs)+1))
    MODEL_DIR = MODEL_DIR+"/run"+str(max(dirs)+1)
else:
  os.mkdir(MODEL_DIR+"/run0")
  MODEL_DIR = MODEL_DIR+"/run0"

if os.path.exists(MODEL_DIR)==False:
  os.mkdir(MODEL_DIR)

In [0]:
session_id = 1
INPUT_ROOT = PATH+'/input_mix/test'
CLEANED_DIR = '/cleaned'
PROCESSED_DIR = '/processed_window_hs_seq2seq_'+ str(configuration['input_sequence_len'])+ '_'+str(configuration['output_sequence_len'])

validation_dataset = Seq2SeqDataSet(INPUT_ROOT+'/validation', configuration['input_sequence_len'], configuration['output_sequence_len'])
validation_dataset = validation_dataset.shuffle()
validation_dataloader = DataLoader(validation_dataset,batch_size=configuration['validation_batch_size'])

encoder = RNNEncoder(rnn_num_layers=1, input_feature_len=configuration['feature_len'], sequence_len=configuration['input_sequence_len'], hidden_size=100, bidirectional=False)
decoder_cell = AttentionDecoderCell(input_feature_len = 1, hidden_size = 100, sequence_len = configuration['output_sequence_len'], decoder_output_size= 1)

encoder.to(device)
decoder_cell.to(device)

model = EncoderDecoderWrapper(encoder, decoder_cell, output_size=12, teacher_forcing=0.3)

criterion = nn.MSELoss()

dirs = ['/'+f for f in listdir(INPUT_ROOT+'/training') if "cleaned_window_hs" in f]
print(dirs)

for loop in range(50):
  print('loop :', loop)
  i = 0
  for CLEANED_DIR  in dirs:
    PROCESSED_DIR = CLEANED_DIR.split( sep="_")[0]+'_seq2seq_' +str(configuration['input_sequence_len'])+'_'+str(configuration['output_sequence_len'])
    print(INPUT_ROOT+PROCESSED_DIR)
    
    training_dataset = Seq2SeqDataSet(INPUT_ROOT+'/training', configuration['input_sequence_len'], configuration['input_sequence_len'])
    training_dataset = training_dataset.shuffle()
    training_dataset = training_dataset[:1000]
    training_dataset = training_dataset[:configuration['training_dataset_length']]
    
    training_dataloader = DataLoader(training_dataset,batch_size=configuration['training_batch_size'])
    model =  train_models( model,learning_rate = 0.001, epochs = 3000,training_dataloader = training_dataloader,id= str(i) )
    torch.save(model.encoder, PATH+'/models_rnn/encoder_latest.model')
    torch.save(model.decoder_cell, PATH+'/models_rnn/encoder_latest.optim')
    i+=1

['/cleaned_window_hs']
loop : 0
/content/drive/My Drive/rubber/input_mix/test/cleaned_seq2seq_12_12
skipping 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


3m 47s (- 1133m 9s) (10 0%) training:0.008325649560227 validation:0.008109897116679
train/validation difference :  -0.00021575244354833135
7m 35s (- 1131m 36s) (20 0%) training:0.007028482950225 validation:0.006688924925295
train/validation difference :  -0.0003395580249296176
11m 27s (- 1133m 34s) (30 1%) training:0.007085892610390 validation:0.006925687300709
train/validation difference :  -0.0001602053096804697
15m 14s (- 1127m 24s) (40 1%) training:0.006626441231921 validation:0.006482445103094
train/validation difference :  -0.00014399612882745605
19m 1s (- 1122m 15s) (50 1%) training:0.006818876319410 validation:0.006807914759824
train/validation difference :  -1.0961559585460738e-05
22m 48s (- 1117m 52s) (60 2%) training:0.006612713559568 validation:0.006489322453791
train/validation difference :  -0.00012339110577616694
26m 36s (- 1113m 54s) (70 2%) training:0.006812821334203 validation:0.006803208933282
train/validation difference :  -9.612400921055024e-06
30m 22s (- 1108m 47s